# Merging of all datasets based on 'PaperId' columns

## Overview
Processing and merging of four different datasets based on the common identifier 'PaperId'. 
The goal is to consolidate information while avoiding duplication of key columns!

---

## Datasets Used:
- **'Initial_df'** → The primary dataset with extracted gene names (starting point for merging).
- **'Cancer_df'** → Contains additional cancer-related information.
- **'Treatment_df'** → Contains treatment-related information.
- **'Study_df'** → Contains study-related information.
- **'Variant_df'** → Contains extarcted variant information.
---

## Steps for merging
1. **Load the datasets** from CSV files.
2. **Identify and ignore redundant columns** in `cancer_df`, `treatment_df`, and `study_df`:
   - Columns like `PaperTitle`, `Author`, `Abstract`, `Citations`, `PubDate`, `PubYear`, `CoFoS`, or `ID` (already in `Initial_df`) are dropped.
3. **Merge all datasets** on `PaperId` using a left join (keeping all records from `Initial_df`).
4. **Check for duplicate column names** after merging to avoid redundancy.
5. **Print a summary** of the final merged dataset, including its shape and column names.

# 1) Set up libraries and datasets

## 1.1) Import libraries and models

In [ ]:
# Import libraries
import os
import time
import datetime
import re
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt

print("Success!")


## 1.2) Load datasets

In [ ]:
# Define directories
base_dir = "/data/BASE_DIRECTORY"
input_dir = os.path.join(base_dir, "Input")
output_dir = os.path.join(base_dir, "Output")
LLM_variant_dir = os.path.join(output_dir, "LLM_variant_analysis")
coassociation_dir = os.path.join(output_dir, "coassociation_analysis")
classifier_dir = os.path.join(output_dir, "gc_batch_files")
variantscape_dir = os.path.join(output_dir, "Variantscape_analysis")

# Confirm current working directory
os.chdir(output_dir)
print("Current directory:", os.getcwd())

# Load datasets
# Input directory files
ESCAT_genes = pd.read_csv(os.path.join(input_dir, "ESCAT_pc_genes.csv"), header=None)
oncomine_genes = pd.read_csv(os.path.join(input_dir, "oncomine_ngs_panel.csv"), header=None)

# Output directory files
initial_df = pd.read_csv(os.path.join(output_dir, "cleaned_BioBERT_data.csv"))
cancer_df = pd.read_csv(os.path.join(output_dir, "binary_cancer_matrix_filtered.csv"))
treatment_df = pd.read_csv(os.path.join(output_dir, "filtered_treatment_mapping_with_matches.csv"))

# Classifier directory file
study_df = pd.read_csv(os.path.join(classifier_dir, "final_gc_classificaton_output_199726.csv"))

# Variant directory file
variant_df = pd.read_csv(os.path.join(LLM_variant_dir, "normalized_merged_variant_matrix_v4.csv"))
print("Success!")

In [ ]:
# Investigate datasets
os.chdir(output_directory)
print("\nDataset Lengths:")
print(f"-Initial dataset: {len(initial_df):,}")
print(f"-Cancer type dataset: {len(cancer_df):,}")
print(f"-Treatment dataset: {len(treatment_df):,}")
print(f"-Study design dataset: {len(study_df):,}")
print(f"-Variant dataset: {len(variant_df):,}")
print("\nNumber of ESCAT genes:", len(ESCAT_genes))
print("Number of oncomine genes:", len(oncomine_genes))

# Check if all datasets have the same length
dataset_lengths = [len(initial_df), len(cancer_df), len(treatment_df), len(study_df),len(variant_df)]

In [ ]:
# Investigate cancer of unkonwn primary origin
column_name = "cancer of unknown primary origin"
if column_name in cancer_df.columns:
    paper_ids_with_unknown_primary = cancer_df[cancer_df[column_name] == 1]['PaperId'].tolist()
    print(f"Found {len(paper_ids_with_unknown_primary)} PaperIds where '{column_name}' is 1.")
    print(f"PaperIds: {paper_ids_with_unknown_primary}")
    matching_treatment_rows = treatment_df[treatment_df['PaperId'].isin(paper_ids_with_unknown_primary)]
    if not matching_treatment_rows.empty:
        print("\nRows in treatment_df with matching PaperIds:")
        print(matching_treatment_rows)

        for column in matching_treatment_rows.columns:
            if column != 'PaperId': 
                if matching_treatment_rows[column].dtype == 'object':
                    count_ones = (matching_treatment_rows[column] == '1').sum()
                else:
                    count_ones = (matching_treatment_rows[column] == 1).sum()
                if count_ones > 0:
                    print(f"Column '{column}' has {count_ones} '1's in matching PaperIds")
        matching_variant_rows = variant_df[variant_df['PaperId'].isin(paper_ids_with_unknown_primary)]

        if not matching_variant_rows.empty:
            print("\nRows in variant_df with matching PaperIds:")
            print(matching_variant_rows)
        else:
            print("\nNo matching PaperIds found in variant_df.")
    else:
        print("\nNo matching PaperIds found in treatment_df.")
else:
    print(f"The column '{column_name}' does not exist in 'cancer_df'.")

# 2) Merging of datasets

In [ ]:
# Ensure all 'PaperId' columns are strings
initial_df["PaperId"] = initial_df["PaperId"].astype(str)
cancer_df["PaperId"] = cancer_df["PaperId"].astype(str)
treatment_df["PaperId"] = treatment_df["PaperId"].astype(str)
study_df["PaperId"] = study_df["PaperId"].astype(str)
variant_df["PaperId"] = variant_df["PaperId"].astype(str)

# Get the set of valid PaperIds from the variant_df
valid_paper_ids = set(variant_df['PaperId'])

# Function to filter a dataset by valid PaperIds
def filter_by_variant_ids(df, df_name):
    if "PaperId" not in df.columns:
        print(f"'PaperId' column not found in {df_name}. Skipping filtering.")
        return df

    initial_count = len(df)
    df_subset = df[df['PaperId'].isin(valid_paper_ids)]
    final_count = len(df_subset)
    
    print(f"{df_name}: Filtered from {initial_count:,} rows to {final_count:,} rows.")
    return df_subset

# Apply filtering to each dataset
initial_df_subset = filter_by_variant_ids(initial_df, "Initial_df")
cancer_df_subset = filter_by_variant_ids(cancer_df, "Cancer_df")
treatment_df_subset = filter_by_variant_ids(treatment_df, "Treatment_df")
study_df_subset = filter_by_variant_ids(study_df, "Study_df")

print("\nFiltered Dataset Lengths:")
print(f"-Initial dataset subset: {len(initial_df_subset):,}")
print(f"-Cancer type dataset subset: {len(cancer_df_subset):,}")
print(f"-Treatment dataset subset: {len(treatment_df_subset):,}")
print(f"-Study design dataset subset: {len(study_df_subset):,}")
print(f"-Variant dataset (reference): {len(variant_df):,}")

In [ ]:
# Convert oncomine_genes to a set for fast lookups
oncomine_genes_set = set(oncomine_genes[0])
columns_to_remove = oncomine_genes_set.union({"Sum_Gene_Mentions"})
cancer_cols_to_drop = [col for col in cancer_df_subset.columns if col in columns_to_remove or "BioBERT" in col]
treatment_cols_to_drop = [col for col in treatment_df_subset.columns if col in columns_to_remove or "BioBERT" in col]

# Drop columns from cancer_df_subset
cancer_df_subset = cancer_df_subset.drop(columns=cancer_cols_to_drop, errors='ignore')

# Drop columns from treatment_df_subset
treatment_df_subset = treatment_df_subset.drop(columns=treatment_cols_to_drop, errors='ignore')
print(f"\nRemoved from cancer_df_subset: {len(cancer_cols_to_drop)} columns")
print(f"Removed from treatment_df_subset: {len(treatment_cols_to_drop)} columns")
print("Columns removed due to:")
print("- Oncomine Genes")
print("- Sum_Gene_Mentions")
print("- Columns containing 'BioBERT'")

In [ ]:
os.chdir(variantscape_directory)
output_file = "merged_detected_variant_df.csv"
initial_row_counts = {
    "initial_df": len(initial_df_subset),
    "cancer_df_rem": len(cancer_df_rem),
    "treatment_df_rem": len(treatment_df_rem),
    "study_df_rem": len(study_df_rem),
    "variant_df_rem": len(variant_df_rem)
}

# Initialize the merged DataFrame with the initial dataset
merged_df = initial_df_subset.copy()
column_sources = {col: "Initial df" for col in initial_df_subset.columns}

datasets_to_merge = [
    (cancer_df_rem, "Cancer"),
    (treatment_df_rem, "Treatment"),
    (study_df_rem, "Study"),
    (variant_df_rem, "Variant")
]

# Merge datasets with progress bar
print("\nStarting the merging process...")
for df, name in tqdm(datasets_to_merge, desc="Merging datasets", unit="dataset"):
    if "PaperId" in df.columns:
        for col in df.columns:
            if col != "PaperId" and col not in column_sources:
                column_sources[col] = name  
        merged_df = pd.merge(merged_df, df, on="PaperId", how="outer", suffixes=("", f"_{name}"))
    else:
        print(f"'PaperId' column missing in {name}, skipping merge.")

# Ensure consistent formatting for numeric columns
print("\nConverting numeric columns...")
numeric_columns = [col for col in merged_df.columns if pd.api.types.is_numeric_dtype(merged_df[col])]

for col in tqdm(numeric_columns, desc="Converting numeric columns", unit="column"):
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0).astype(int)

# Add the source row at the top *after* merging
source_row = pd.Series(column_sources, name='Source_Row')
merged_df = pd.concat([pd.DataFrame([source_row]), merged_df], ignore_index=True)
print("\nFirst 5 rows of merged dataset:")
print(merged_df.head())

# Check for Duplicate Column Names
print("\nChecking for duplicate column names...")
duplicate_columns = merged_df.columns[merged_df.columns.duplicated()].unique()

if len(duplicate_columns) > 0:
    print(f"\nDuplicate column names found ({len(duplicate_columns):,}):")
    for col in tqdm(duplicate_columns, desc="Reporting duplicates", unit="column"):
        print(col)
else:
    print("No duplicate column names found.")

print("\nMerging completed successfully.")
print(f"Final dataset shape: {merged_df.shape[0]:,} rows, {merged_df.shape[1]:,} columns")
print("\nFirst 10 rows of merged dataset:")
print(merged_df.head(10))

In [ ]:
# Print final results
print("\nMerging completed. Final dataset shape:", f"{merged_df.shape[0]:,} rows, {merged_df.shape[1]:,} columns")
print("Columns in final dataset:", merged_df.columns.tolist())
print("\nFirst 10 rows of merged dataset:")
print(merged_df.head(10))

In [ ]:
# Define the columns relevant to all_conditions
columns_of_interest = [
    "Sum_Gene_Mentions", "Cancer_Type_Sum", "Sum_treatments", "total_variant_count", "Study_design"
]
print("\nFirst 5 rows of merged_df with relevant columns:")
print(merged_df[columns_of_interest].head())
print(merged_df.shape)

In [ ]:
# Replace 'Cancer_of_Primary_Unknown' with the actual column name
column_name = "cancer of unknown primary origin"
if column_name in merged_df.columns:
    count_1 = (merged_df[column_name] == 1).sum() 
    count_0 = (merged_df[column_name] == 0).sum() 

    print(f"Count of '1' in '{column_name}': {count_1}")
    print(f"Count of '0' in '{column_name}': {count_0}")
else:
    print(f"The column '{column_name}' does not exist in the DataFrame.")

In [ ]:
# Save dataset as CSV with progress bar (as it is huge, this takes about 1h)
os.chdir(variantscape_directory)
file_name = "final_merged_df.csv"
chunk_size = 5_000  
total_rows = len(merged_df)

with open(file_name, "w", encoding="utf-8", newline='') as f:
    merged_df.head(0).to_csv(f, index=False) 
    
    # Progress bar for writing rows in chunks
    with tqdm(total=total_rows, desc="Saving CSV", unit=" rows", bar_format="{l_bar}{bar} {n_fmt}/{total_fmt} [{elapsed}<{remaining}]") as pbar:
        for i in range(0, total_rows, chunk_size):
            chunk = merged_df.iloc[i:i + chunk_size]
            chunk.to_csv(f, index=False, header=False, mode='a')
            pbar.update(len(chunk))

print("\nMerged dataframe successfully saved.")
print(f"Total Rows: {total_rows:,}")
print(f"Total Columns: {len(merged_df.columns):,}")
print(f"File Saved: {file_name}")

In [ ]:
# Ensure numeric types (without modifying anything)
cols_to_check = ["Sum_Gene_Mentions", "Cancer_Type_Sum", "Sum_treatments", "total_variant_count"]

# Count rows where value is >= 1 for each column, ignoring index 0
counts_ge1 = (merged_df.iloc[1:][cols_to_check] >= 1).sum()

# Count non-empty 'Study_design' values (not null, not '0', not numeric 0), excluding index 0
study_design_non_empty = merged_df.iloc[1:]["Study_design"].notna() & (merged_df.iloc[1:]["Study_design"] != "0") & (merged_df.iloc[1:]["Study_design"] != 0)
count_study_design = study_design_non_empty.sum()
all_conditions = (
    (merged_df.iloc[1:]["Sum_Gene_Mentions"] >= 1) &
    (merged_df.iloc[1:]["Cancer_Type_Sum"] >= 1) &
    (merged_df.iloc[1:]["Sum_treatments"] >= 1) &
    (merged_df.iloc[1:]["total_variant_count"] >= 1) &
    study_design_non_empty
)
count_all_conditions = all_conditions.sum()
print("\nNumber of rows with values ≥ 1 (ignoring index 0):")
for col, count in counts_ge1.items():
    print(f"- {col}: {count:,}")

print(f"- Study_design (not empty or 0): {count_study_design:,}")
print(f"- All conditions TRUE (ignoring index 0): {count_all_conditions:,}")

In [ ]:
# Combine all conditions (ignoring index 0)
all_conditions = (
    (merged_df.iloc[1:]["Sum_Gene_Mentions"] >= 1) &
    (merged_df.iloc[1:]["Cancer_Type_Sum"] >= 1) &
    (merged_df.iloc[1:]["Sum_treatments"] >= 1) &
    (merged_df.iloc[1:]["total_variant_count"] >= 1)
)

# Get the subset of rows where all conditions are true, ignoring index 0
filtered_subset = merged_df.iloc[1:][all_conditions].copy()
if 'PaperId' in filtered_subset.columns:
    print("The column 'PaperId' is present in the filtered DataFrame.")
else:
    print("The column 'PaperId' is not present in the filtered DataFrame.")
filtered_subset_with_index_0 = pd.concat([merged_df.iloc[[0]], filtered_subset])
print(f"\nShape of the filtered DataFrame (with index 0): {filtered_subset_with_index_0.shape}")
print(filtered_subset_with_index_0.head())
print(f"Final shape of the filtered DataFrame: {filtered_subset_with_index_0.shape}")
filtered_subset_with_index_0.to_csv('filtered_subset_with_index_0.csv', index=False)
print("Filtered DataFrame has been saved as 'filtered_subset_with_index_0.csv'.")

In [ ]:
# Original dataset lengths for reference
original_lengths = {
    "Initial Dataset": len(initial_df),
    "Cancer Type Dataset": len(cancer_df),
    "Study Design Dataset": len(study_df),
    "Treatment Dataset": len(treatment_df),
    "Variant Dataset": len(variant_df)
}

print("\nOriginal Dataset Lengths (with percentage of initial dataset):")
for name, length in original_lengths.items():
    percentage = (length / original_lengths["Initial Dataset"]) * 100
    print(f"- {name}: {length:,} ({percentage:.2f}%)")

percentage_all_conditions = (count_all_conditions / original_lengths["Initial Dataset"]) * 100
print(f"\n- All conditions TRUE: {count_all_conditions:,} ({percentage_all_conditions:.2f}%)")

labels = [
    "Initial Dataset", "Cancer Type Dataset", "Study Design Dataset", 
    "Treatment Dataset", "Variant Dataset", "All conditions TRUE"
]

counts = list(original_lengths.values()) + [count_all_conditions]
percentages = [(count / original_lengths["Initial Dataset"]) * 100 for count in counts]
dataset_lengths_df = pd.DataFrame({
    "Category": labels,
    "Count": counts,
    "Percentage": percentages
})

csv_file_path = "conditional_filterting_of_final_datasets_for_coassciaitions_analysis.csv"
dataset_lengths_df.to_csv(csv_file_path, index=False)
print(f"\nData successfully saved to: {csv_file_path}")

## Create figure

In [ ]:
# Original dataset lengths for reference
original_lengths = {
    "Initial Dataset": len(initial_df),
    "Cancer Type Dataset": len(cancer_df),
    "Treatment Dataset": len(treatment_df),
    "Study Design Dataset": len(study_df),
    "Variant Dataset": len(variant_df)
}

# Counts from original datasets
count_1 = original_lengths["Initial Dataset"]
count_2 = original_lengths["Cancer Type Dataset"]
count_3 = original_lengths["Study Design Dataset"]
count_4 = original_lengths["Treatment Dataset"]
count_5 = original_lengths["Variant Dataset"]
count_all_conditions = count_all_conditions

labels = [
    "Extracted genes",
    "Extracted cancer types",
    "Classified study design", 
    "Extracted treatments",   
    "Identified variants",
    "All conditions TRUE"
]

counts = [count_1, count_2, count_3, count_4, count_5, count_all_conditions]
percentages = [(count / count_1 * 100) if count_1 > 0 else 0 for count in counts]

# Plotting the bar chart
fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(labels, counts, color="#0073e6", edgecolor='black', alpha=0.95) 

for bar, count, pct in zip(bars, counts, percentages):
    label = f"{count:,} ({pct:.1f}%)"
    ax.text(bar.get_x() + bar.get_width() / 2,
            bar.get_height() + max(counts) * 0.01,
            label,
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_title("Dataset Sizes and Condition Filtering (With Percentages)", fontsize=16, fontweight='bold', pad=20)
ax.set_ylabel("Number of Rows", fontsize=14, fontweight='bold')
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels, fontsize=12, ha="right", rotation=15)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# Data inspection and export of filtered dataset

first_row = filtered_subset_with_index_0.iloc[0]
unique_values = first_row.unique()
print("Unique values in the first row:")
print(unique_values)

# Find columns where the value is NaN
nan_columns = first_row[first_row.isna()].index.tolist()
print("\n\nColumns with NaN in the first row:")
for col in nan_columns:
    print(f"- {col}")

columns_of_interest = ["Sum_Gene_Mentions", "Cancer_Type_Sum", "Sum_treatments", "total_variant_count"]
print("Rows from selected columns:")
print(filtered_subset_with_index_0[columns_of_interest].head(10))

# Save the updated and filtered dataset to CSV
merged_detected_variant_df=filtered_subset_with_index_0.copy()
output_file = "merged_detected_variant_df.csv"
merged_detected_variant_df.to_csv(output_file, index=False)
print(f"Filtered dataset saved as '{output_file}'")

rows_before_col_drop=len(merged_detected_variant_df)
col_before_col_drop=len(merged_detected_variant_df.columns)
print(f"Total rows: {rows_before_col_drop:,}")
print(f"Total columns: {col_before_col_drop:,}")

In [ ]:
## Drop the COLUMNS as well where the variant sum is ZERO!

print(merged_detected_variant_df.columns.tolist())

In [ ]:
# Load or use existing dataframe
if 'merged_detected_variant_df' not in globals():
    merged_detected_variant_df = pd.read_csv("merged_detected_variant_df.csv")

# Drop 'Unnamed: 0' if it exists — this was likely the index when saved
if "Unnamed: 0" in merged_detected_variant_df.columns:
    print("Dropping 'Unnamed: 0' — it was likely saved as index.")
    merged_detected_variant_df = merged_detected_variant_df.drop(columns=["Unnamed: 0"])

# Copy for processing
rem_col_variant_df = merged_detected_variant_df.copy()

# Manually defined columns to ignore
ignore_columns = {
    'PaperId', 'PaperTitle', 'Citations', 'CoFoS', 'Authors', 'Abstract', 
    'Language', 'PubYear', 'PubDate', 'BioBERT', 'Sum_Gene_Mentions', 
    'Extracted_Cancer_Terms_old', 'Extracted_Cancer_Terms','Cancer_Type_Sum',
    'Mapped_Cancer_Terms', 'Unmatched_Cancer_Terms', 'Remapped_Cancer_Terms', 
    'Final_Mapped_Cancer_Terms', 'Treatment_matches', 'Sum_treatments', 'Study_design',
    'total_variant_count','LLM_Prompt', 'LLM_Response', 'Cleaned_Variant_Gene_Pairs'
}

# Additional prefixes to ignore
ignore_prefixes = ('Gene_', 'Variant_', 'LLM_Prompt', 'LLM_Response')

# Add any column starting with those prefixes to the ignore list
for col in rem_col_variant_df.columns:
    if col.startswith(ignore_prefixes):
        ignore_columns.add(col)

# Identify candidate binary columns (not ignored)
candidate_cols = [col for col in rem_col_variant_df.columns if col not in ignore_columns]

# Exclude row 0 (metadata row)
data_for_analysis = rem_col_variant_df.iloc[1:].copy()

# Convert to numeric and normalize values
data_for_analysis[candidate_cols] = data_for_analysis[candidate_cols].apply(pd.to_numeric, errors='coerce')
data_for_analysis[candidate_cols] = data_for_analysis[candidate_cols].astype('Int64')
non_binary_columns = []
print("Checking for non-binary values (0/1 only)...")
for col in tqdm(candidate_cols, desc="Validating columns", unit="col"):
    unique_vals = data_for_analysis[col].dropna().unique()
    if not all(val in [0, 1] for val in unique_vals):
        non_binary_columns.append((col, unique_vals.tolist()))
if non_binary_columns:
    print(f"{len(non_binary_columns)} column(s) (excluding ignored and row 0) contain values other than 0 or 1.")
    print("First 5:")
    for col, vals in non_binary_columns[:5]:
        print(f"- {col}: {vals}")
else:
    print("All candidate columns contain only 0s and 1s (excluding row 0).")

In [ ]:
# Get all binary-valid columns by removing the ones with non-binary values
binary_columns = [col for col in candidate_cols if col not in [c[0] for c in non_binary_columns]]
print(f"\nBinary columns (total: {len(binary_columns)}):\n")
for col in binary_columns:
    print(col)

In [ ]:
# Drop 0 columns
# Only proceed if all binary columns are valid
if not non_binary_columns:
    steps = [
        "Check which binary columns are all-zero (excluding row 0)",
        "Print summary",
        "Drop all-zero binary columns",
        "Save cleaned DataFrame"
    ]
    pbar = tqdm(total=len(steps), desc="Processing", bar_format='{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}]')
    col_sums = data_for_analysis[candidate_cols].sum()
    cols_all_zero = col_sums[col_sums == 0].index.tolist()
    cols_with_ones = col_sums[col_sums > 0].index.tolist()
    pbar.update(1)
    total = len(candidate_cols)
    num_zero = len(cols_all_zero)
    num_one = len(cols_with_ones)
    pct_one = (num_one / total * 100) if total > 0 else 0
    pct_zero = (num_zero / total * 100) if total > 0 else 0

    print("\nColumn Summary (excluding metadata and row 0):")
    print(f"- Columns with at least one '1': {num_one:,} ({pct_one:.1f}%)")
    print(f"- Columns with only '0's:         {num_zero:,} ({pct_zero:.1f}%)")
    print(f"- Dropped columns: {num_zero:,}")
    print("  First 10 dropped columns:", cols_all_zero[:10])
    with open("dropped_zero_columns.txt", "w") as f:
        for col in cols_all_zero:
            f.write(col + "\n")
    print("Full list of dropped columns saved to 'dropped_zero_columns.txt'")
    pbar.update(1)

    small_rem_col_variant_df = rem_col_variant_df.drop(columns=cols_all_zero)
    pbar.update(1)
    output_file = "small_rem_col_variant_df.csv"
    small_rem_col_variant_df.to_csv(output_file, index=False)
    print(f"\nFiltered dataset saved as '{output_file}'")
    pbar.update(1)

    pbar.close()
else:
    print("Non-binary columns detected.")

In [ ]:
# Get shape before column dropping
rows_before_col_drop, col_before_col_drop = rem_col_variant_df.shape
# Get shape after column dropping
rows_after_col_drop, col_after_col_drop = small_rem_col_variant_df.shape
print("\n=== Final Dataset Summary ===")
print(f"{'Metric':<30} {'Before':>15} {'After':>15}")
print(f"{'Total rows':<30} {rows_before_col_drop:>15,} {rows_after_col_drop:>15,}")
print(f"{'Total columns':<30} {col_before_col_drop:>15,} {col_after_col_drop:>15,}")
print(f"{'Total columns dropped':<30} {'':>15} {col_before_col_drop - col_after_col_drop:>15,}")

In [ ]:
# Drop columns starting with Variant_ or Gene_
variant_gene_cols = [col for col in small_rem_col_variant_df.columns if col.startswith(("Variant_", "Gene_"))]
final_variant_df_for_analysis = small_rem_col_variant_df.drop(columns=variant_gene_cols)

ordered_front = [col for col in final_variant_df_for_analysis.columns if col in ignore_columns]
remaining_cols = [col for col in final_variant_df_for_analysis.columns if col not in ordered_front]
final_variant_df_for_analysis = final_variant_df_for_analysis[ordered_front + remaining_cols]
final_variant_df_for_analysis.to_csv("final_variant_df_for_analysis.csv", index=False)
original_col_count = small_rem_col_variant_df.shape[1]
dropped_col_count = len(variant_gene_cols)
remaining_col_count = final_variant_df_for_analysis.shape[1]

print("\n=== Final Variant Dataset Shape ===")
print(f"{'Metric':<30} {'Value':>20}")
print("-" * 50)
print(f"{'Total rows':<30} {final_variant_df_for_analysis.shape[0]:>20,}")
print(f"{'Original columns':<30} {original_col_count:>20,}")
print(f"{'Columns dropped':<30} {dropped_col_count:>20,}")
print(f"{'Remaining columns':<30} {remaining_col_count:>20,}")

if original_col_count - dropped_col_count == remaining_col_count:
    print("Column count check: MATCH")
else:
    print("Column count check: MISMATCH")

In [ ]:
# print(small_rem_col_variant_df)
# print(final_variant_df_for_analysis.columns.tolist())
print("\n=== Columns in final_variant_df_for_analysis ===")
for col in final_variant_df_for_analysis.columns:
    print(col)

In [ ]:
# Dataset to use: final_variant_df_for_analysis.csv